In [1]:
from pyevolve import G1DList, GAllele
from pyevolve import GSimpleGA
from pyevolve import Mutators
from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import DBAdapters
from pyevolve import Selectors
from geopy.distance import geodesic

import math

import sys, random, os
random.seed(1024)
from math import sqrt

PIL_SUPPORT = None

try:
    from PIL import Image, ImageDraw, ImageFont
    PIL_SUPPORT = True
except ImportError:
    PIL_SUPPORT = False


cm     = []
R = 6371000; # metres
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]
CITIES = len(LAT)
coords = list(zip(LAT, LON))
WIDTH   = 1024
HEIGHT  = 768
LAST_SCORE = -1

In [2]:
def cartesian_matrix(coords):
    """ A distance matrix """
    matrix={}
    for i,(x1,y1) in enumerate(coords):
        for j,(x2,y2) in enumerate(coords):
            dx, dy = x1-x2, y1-y2
            dist=sqrt(dx*dx + dy*dy)
            matrix[i,j] = dist
    return matrix

In [3]:
def lat_long_cartesian_matrix(coords):
    """ A distance matrix """
    
    matrix={}
    for i, town1 in enumerate(coords):
        for j, town2 in enumerate(coords):
            #dlat, dlong = math.radians(lat2-lat1), math.radians(long1-long2)
            #latRad1 = math.radians(lat1)
            #latRad2 = math.radians(lat2)
            #a = math.sin(dlat/2) ** 2 + math.cos(latRad1) * math.cos(latRad2) * math.sin(dlong/2) ** 2;
            #c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
            #d = R * c;
            matrix[i,j] = geodesic(town1, town2).km
    return matrix

In [4]:
def tour_length(matrix, tour):
    """ Returns the total length of the tour """
    total = 0
    t = tour.getInternalList()
    for i in range(CITIES):
        j      = (i+1)%CITIES
        total += matrix[t[i], t[j]]
    return total

In [5]:
def G1DListTSPInitializator(genome, **args):
    """ The initializator for the TSP (We just shuffle the list) """
    lst = [i for i in range(genome.getListSize())]
    random.shuffle(lst)
    genome.setInternalList(lst)

In [6]:
def current_best(ga_engine):
    # Here you have access to the GA Engine
    best = ga.bestIndividual()
    for i in range(0,len(best)):
        print(chr(best[i]))
    return False

In [12]:
global cm, LAT, LON, WIDTH, HEIGHT

cm     = lat_long_cartesian_matrix(coords)
genome = G1DList.G1DList(len(coords))

genome.evaluator.set(lambda chromosome: tour_length(cm, chromosome))
genome.crossover.set(Crossovers.G1DListCrossoverEdge)
genome.initializator.set(G1DListTSPInitializator)
genome.mutator.set(Mutators.G1DListMutatorSwap)

# 3662.69
ga = GSimpleGA.GSimpleGA(genome)
ga.setGenerations(500) # default 100
ga.setMinimax(Consts.minimaxType["minimize"])
ga.setCrossoverRate(0.9) #default 0.9
ga.setMutationRate(0.1) # default 0.02
ga.setPopulationSize(80) # default 80
ga.selector.set(Selectors.GTournamentSelector) # default Selectors.GRankSelector
ga.setElitism(True)


# Permet de ploter les jolis graphes via la commande python pyevolve_graph.py -0 -i run1
dbadapter = DBAdapters.DBSQLite(identify="run1")
ga.setDBAdapter(dbadapter)

# This is to make a video

ga.evolve(freq_stats=10)

best = ga.bestIndividual()

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8076.43(8699.12)/5677.08(5189.84)/6730.36(6730.36)]
Gen. 10 (2.00%): Max/Min/Avg Fitness(Raw) [7411.86(7622.70)/4542.61(4263.73)/6176.55(6176.55)]
Gen. 20 (4.00%): Max/Min/Avg Fitness(Raw) [7433.20(8050.15)/4690.09(3940.98)/6194.33(6194.33)]
Gen. 30 (6.00%): Max/Min/Avg Fitness(Raw) [7240.31(8107.20)/4815.81(3940.98)/6033.59(6033.59)]
Gen. 40 (8.00%): Max/Min/Avg Fitness(Raw) [7082.65(7669.54)/4592.26(3940.98)/5902.21(5902.21)]
Gen. 50 (10.00%): Max/Min/Avg Fitness(Raw) [7369.74(7308.78)/3826.06(3940.98)/6141.45(6141.45)]
Gen. 60 (12.00%): Max/Min/Avg Fitness(Raw) [7393.57(7520.74)/4148.68(3940.98)/6161.31(6161.31)]
Gen. 70 (14.00%): Max/Min/Avg Fitness(Raw) [7258.11(7624.23)/4430.63(3940.98)/6048.42(6048.42)]
Gen. 80 (16.00%): Max/Min/Avg Fitness(Raw) [7202.92(7322.92)/4072.78(3879.88)/6002.43(6002.43)]
Gen. 90 (18.00%): Max/Min/Avg Fitness(Raw) [7298.80(7562.00)/3992.17(3539.93)/6082.33(6082.33)]
Gen. 100 (20.00%): Max/Min/Avg Fitness(Raw) [7

In [15]:
print(best)

- GenomeBase
	Score:			 3346.761974
	Fitness:		 4499.965245

	Params:		 {}

	Slot [Evaluator] (Count: 1)
		Name: <lambda> - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: G1DListTSPInitializator - Weight: 0.50
		Doc:  The initializator for the TSP (We just shuffle the list) 
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  The mutator of G1DList, Swap Mutator

   .. note:: this mutator is :term:`Data Type Independent`

   
	Slot [Crossover] (Count: 1)
		Name: G1DListCrossoverEdge - Weight: 0.50
		Doc:  THe Edge Recombination crossover for G1DList (widely used for TSP problem)

   See more information in the `Edge Recombination Operator <http://en.wikipedia.org/wiki/Edge_recombination_operator>`_
   Wikipedia entry.
   

- G1DList
	List size:	 14
	List:		 [0, 1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9]




Lien vers Overleaf : https://www.overleaf.com/16932456chzcxjsfykqs